In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import time
import copy
import string
import json
import pickle as pk
import math

from treelib import Node, Tree

from collections import Counter

from math import sin, asin, cos, radians, fabs, sqrt

In [2]:
############################    从文件中 读取 数据   ##############
def read_q(filename):
    f = open(filename,"rb+")
    st = pk.load(f)
    f.close()
    return st

In [3]:
##########################    向文件中  写 数据  #####################
def write_q(st, filename):
    f = open(filename, "wb+")
    pk.dump(st,f)
    f.close()

In [4]:
################################  停留点 ----> 转换轨迹段 到 停留点  #############
# 计算停留点 stay_point 的平均地理位置 及 停留时间
# input:  一个 stay_point 轨迹段，       所有位置 及 访问时刻  [[Lat, Long, Time],[],[]]
# output:  这个stay_point 的平均地理位置 及 停留时长 [Lat, Long, Duration]   单位， 分钟/min
def mean_point(a):
    b = 0
    c = 0
    for i in range(0, len(a)):
        b = b + a[i][0]
        c = c + a[i][1]
    
    b = b / len(a)
    c = c / len(a)
    
    d = (a[len(a)-1][2] -a[0][2]) * 24*60 
    
    return [b, c, d]

In [5]:
#######################   Dis()  根据经纬度计算距离 distance(a)  ####################3
#     input:  a  2*2 两个点的经纬度坐标；
#     output:  距离  m

################    测试用例   #################
#a = [[40.009532,116.322153],[40.009493,116.322103]]
#b = pd.DataFrame(a)
#Dis(b)

EARTH_RADIUS = 6371.004           #地球平均半径，6371km
 
def hav(theta):
    s = sin(theta / 2)
    return s * s
 
def Dis(a): 
#def get_distance_hav(lat0, lng0, lat1, lng1):
    """用haversine公式计算球面两点间的距离。"""
    # 经纬度转换成弧度
    lat0 = radians(a.iloc[0,0])
    lat1 = radians(a.iloc[1,0])
    lng0 = radians(a.iloc[0,1])
    lng1 = radians(a.iloc[1,1])
 
    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    dist = 2 * EARTH_RADIUS * asin(sqrt(h))*1000
 
    return dist

In [6]:
###################################   单停留点与 POI的匹配    ############
# 输入:  停留点list (一个 三元组) , st_poi_dic 字典list  [{},{},{}]
# 输出： 最近的POI  及 停留时间  和 语义类型    五元组[id, lat,lng,time,category]
#st_p = st_p_list[0]

def match(st_p, sp_dic):
    poi_st_1 = [] 
    for i in range(0, len(sp_dic)):
        a = [[sp_dic[i]["location"]["lat"], sp_dic[i]["location"]["lng"]],[st_p[0],st_p[1]]]
        b = pd.DataFrame(a)
        c = Dis(b)
        poi_st_1.append(c)    
    #print(len(poi_st_1)) 
    j = poi_st_1.index( min(poi_st_1) )
    return  [sp_dic[j]["id"],sp_dic[j]["location"]["lat"], sp_dic[j]["location"]["lng"], st_p[2],sp_dic[j]["category"]]
    

In [4]:
####################################   读 st_p_1/2 轨迹  ###############################
# 输出 st_p_1, st_p_2  为原始停留点数据-轨迹   四重list

file_name_st1 = ".\\Stay_point\\st_p_1-20190726-120655.txt"        # 读取第一类停留点
b = open(file_name_st1, "r+")
out = b.read()
st_p_1 =  json.loads(out)
b.close()

file_name_st2 = ".\\Stay_point\\st_p_2-20190726-120655.txt"        # 读取第二类停留点
b = open(file_name_st2, "r+")
out = b.read()
st_p_2 =  json.loads(out)
b.close()

len(st_p_2[0])

171

In [22]:
##########################  均值后的 st_pa list 所有用户   ###########################
## 输入 st_p_1\ 2
## 输出 均值后的 st_pa list  三层list  [user][file][stop]

filename = ".\\os_file\\st_pa.pkl"
#filename2 = ".\\os_file\\st_pa_2.pkl"

st_pa = []    #  停留点的list 

m1 = st_p_1
m2 = st_p_2
b = 0
for i in range(0, len(m2)):        
             # 对 m2[i]  第i个用户，操作文件  user_files
    st_p_u = []
    a1 = b
    print("第：",i,"个用户")
    if len(m2[i]) == 0:            #  如果  user_file  的文件数为 0，  len(st_p_1[i])是用户的文件数
        continue
    else:
               
        for j in range(0, len(m2[i])):
            # 对用户i的文件j, st_p[i][j],     文件
            print()    
            st_p_f = []
            print("      第：",j,"个文件")
            if len(m2[i][j]) != 0:       #  文件内没有stay point，
                for k in range(0, len(m2[i][j])):
                    # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                    #st_p_1.append(st_p_1_txt[i][j][k])
                    if len(m2[i][j][k]) ==0:
                        continue
                    else:                # 操作每一个 st_p_1_txt[i][j][k] 停留点
                        
                        #此处定义一个 POI
                        a = []
                        a = mean_point(m2[i][j][k])
                        #print(a)
                        st_p_f.append(a)
                        
            
            if len(m1[i][j]) != 0:       #  文件内有stay point，   
                for l in range(0, len(m1[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                    #st_p_1.append(st_p_1_txt[i][j][k])
                    if len(m1[i][j][l]) ==0:
                        continue
                    else:                # 操作每一个 st_p_1_txt[i][j][k] 停留点
                        
                        #此处定义一个 POI
                        a = []
                        a = mean_point(m1[i][j][l])
                        #print(a)
                        st_p_f.append(a)
                        
            b += len(st_p_f)
            print("       第",j,"个文件共有停留点：",len(st_p_f),"个")
            st_p_u.append(st_p_f)
    print("第:",i,"用户共有停留点：",b-a1,"个",)        
    st_pa.append(st_p_u)

print("共有停留点： ",b,"个")

write_q(st_pa, filename)
#filename1 = ".\\os_file\\st_p.csv"
#write_st_p(st_p, filename1)

第： 0 个用户

      第： 0 个文件
       第 0 个文件共有停留点： 2 个

      第： 1 个文件
       第 1 个文件共有停留点： 1 个

      第： 2 个文件
       第 2 个文件共有停留点： 0 个

      第： 3 个文件
       第 3 个文件共有停留点： 0 个

      第： 4 个文件
       第 4 个文件共有停留点： 1 个

      第： 5 个文件
       第 5 个文件共有停留点： 0 个

      第： 6 个文件
       第 6 个文件共有停留点： 0 个

      第： 7 个文件
       第 7 个文件共有停留点： 0 个

      第： 8 个文件
       第 8 个文件共有停留点： 4 个

      第： 9 个文件
       第 9 个文件共有停留点： 0 个

      第： 10 个文件
       第 10 个文件共有停留点： 1 个

      第： 11 个文件
       第 11 个文件共有停留点： 3 个

      第： 12 个文件
       第 12 个文件共有停留点： 0 个

      第： 13 个文件
       第 13 个文件共有停留点： 0 个

      第： 14 个文件
       第 14 个文件共有停留点： 3 个

      第： 15 个文件
       第 15 个文件共有停留点： 0 个

      第： 16 个文件
       第 16 个文件共有停留点： 1 个

      第： 17 个文件
       第 17 个文件共有停留点： 0 个

      第： 18 个文件
       第 18 个文件共有停留点： 0 个

      第： 19 个文件
       第 19 个文件共有停留点： 0 个

      第： 20 个文件
       第 20 个文件共有停留点： 0 个

      第： 21 个文件
       第 21 个文件共有停留点： 0 个

      第： 22 个文件
       第 22 个文件共有停留点： 0 个

      第： 23 个文件
     

       第 107 个文件共有停留点： 1 个

      第： 108 个文件
       第 108 个文件共有停留点： 3 个

      第： 109 个文件
       第 109 个文件共有停留点： 4 个

      第： 110 个文件
       第 110 个文件共有停留点： 0 个

      第： 111 个文件
       第 111 个文件共有停留点： 0 个

      第： 112 个文件
       第 112 个文件共有停留点： 0 个

      第： 113 个文件
       第 113 个文件共有停留点： 0 个

      第： 114 个文件
       第 114 个文件共有停留点： 0 个

      第： 115 个文件
       第 115 个文件共有停留点： 0 个

      第： 116 个文件
       第 116 个文件共有停留点： 0 个

      第： 117 个文件
       第 117 个文件共有停留点： 0 个

      第： 118 个文件
       第 118 个文件共有停留点： 0 个

      第： 119 个文件
       第 119 个文件共有停留点： 0 个

      第： 120 个文件
       第 120 个文件共有停留点： 1 个

      第： 121 个文件
       第 121 个文件共有停留点： 4 个

      第： 122 个文件
       第 122 个文件共有停留点： 3 个

      第： 123 个文件
       第 123 个文件共有停留点： 3 个

      第： 124 个文件
       第 124 个文件共有停留点： 0 个

      第： 125 个文件
       第 125 个文件共有停留点： 0 个

      第： 126 个文件
       第 126 个文件共有停留点： 1 个

      第： 127 个文件
       第 127 个文件共有停留点： 1 个

      第： 128 个文件
       第 128 个文件共有停留点： 0 个

      第： 129 个文件
       第 1

       第 284 个文件共有停留点： 2 个

      第： 285 个文件
       第 285 个文件共有停留点： 1 个

      第： 286 个文件
       第 286 个文件共有停留点： 0 个

      第： 287 个文件
       第 287 个文件共有停留点： 4 个

      第： 288 个文件
       第 288 个文件共有停留点： 11 个

      第： 289 个文件
       第 289 个文件共有停留点： 2 个

      第： 290 个文件
       第 290 个文件共有停留点： 3 个

      第： 291 个文件
       第 291 个文件共有停留点： 0 个

      第： 292 个文件
       第 292 个文件共有停留点： 2 个

      第： 293 个文件
       第 293 个文件共有停留点： 6 个

      第： 294 个文件
       第 294 个文件共有停留点： 2 个

      第： 295 个文件
       第 295 个文件共有停留点： 2 个

      第： 296 个文件
       第 296 个文件共有停留点： 0 个

      第： 297 个文件
       第 297 个文件共有停留点： 9 个

      第： 298 个文件
       第 298 个文件共有停留点： 4 个

      第： 299 个文件
       第 299 个文件共有停留点： 6 个

      第： 300 个文件
       第 300 个文件共有停留点： 1 个

      第： 301 个文件
       第 301 个文件共有停留点： 5 个

      第： 302 个文件
       第 302 个文件共有停留点： 4 个

      第： 303 个文件
       第 303 个文件共有停留点： 5 个

      第： 304 个文件
       第 304 个文件共有停留点： 0 个

      第： 305 个文件
       第 305 个文件共有停留点： 4 个

      第： 306 个文件
       第 

       第 314 个文件共有停留点： 4 个

      第： 315 个文件
       第 315 个文件共有停留点： 5 个

      第： 316 个文件
       第 316 个文件共有停留点： 6 个

      第： 317 个文件
       第 317 个文件共有停留点： 0 个

      第： 318 个文件
       第 318 个文件共有停留点： 3 个

      第： 319 个文件
       第 319 个文件共有停留点： 2 个

      第： 320 个文件
       第 320 个文件共有停留点： 0 个

      第： 321 个文件
       第 321 个文件共有停留点： 1 个

      第： 322 个文件
       第 322 个文件共有停留点： 0 个

      第： 323 个文件
       第 323 个文件共有停留点： 6 个

      第： 324 个文件
       第 324 个文件共有停留点： 3 个

      第： 325 个文件
       第 325 个文件共有停留点： 0 个

      第： 326 个文件
       第 326 个文件共有停留点： 0 个

      第： 327 个文件
       第 327 个文件共有停留点： 3 个

      第： 328 个文件
       第 328 个文件共有停留点： 0 个

      第： 329 个文件
       第 329 个文件共有停留点： 4 个

      第： 330 个文件
       第 330 个文件共有停留点： 1 个

      第： 331 个文件
       第 331 个文件共有停留点： 3 个

      第： 332 个文件
       第 332 个文件共有停留点： 7 个

      第： 333 个文件
       第 333 个文件共有停留点： 2 个

      第： 334 个文件
       第 334 个文件共有停留点： 0 个

      第： 335 个文件
       第 335 个文件共有停留点： 0 个

      第： 336 个文件
       第 3

       第 15 个文件共有停留点： 0 个

      第： 16 个文件
       第 16 个文件共有停留点： 0 个

      第： 17 个文件
       第 17 个文件共有停留点： 0 个

      第： 18 个文件
       第 18 个文件共有停留点： 0 个

      第： 19 个文件
       第 19 个文件共有停留点： 0 个

      第： 20 个文件
       第 20 个文件共有停留点： 0 个

      第： 21 个文件
       第 21 个文件共有停留点： 0 个

      第： 22 个文件
       第 22 个文件共有停留点： 0 个

      第： 23 个文件
       第 23 个文件共有停留点： 0 个

      第： 24 个文件
       第 24 个文件共有停留点： 0 个

      第： 25 个文件
       第 25 个文件共有停留点： 0 个

      第： 26 个文件
       第 26 个文件共有停留点： 0 个

      第： 27 个文件
       第 27 个文件共有停留点： 0 个

      第： 28 个文件
       第 28 个文件共有停留点： 0 个

      第： 29 个文件
       第 29 个文件共有停留点： 0 个

      第： 30 个文件
       第 30 个文件共有停留点： 0 个

      第： 31 个文件
       第 31 个文件共有停留点： 0 个

      第： 32 个文件
       第 32 个文件共有停留点： 1 个

      第： 33 个文件
       第 33 个文件共有停留点： 0 个

      第： 34 个文件
       第 34 个文件共有停留点： 0 个

      第： 35 个文件
       第 35 个文件共有停留点： 0 个

      第： 36 个文件
       第 36 个文件共有停留点： 0 个

      第： 37 个文件
       第 37 个文件共有停留点： 0 个

      第： 38 个文件
       第 38

...

      第： 633 个文件
       第 633 个文件共有停留点： 0 个

      第： 634 个文件
       第 634 个文件共有停留点： 0 个

      第： 635 个文件
       第 635 个文件共有停留点： 0 个

      第： 636 个文件
       第 636 个文件共有停留点： 0 个

      第： 637 个文件
       第 637 个文件共有停留点： 0 个

      第： 638 个文件
       第 638 个文件共有停留点： 2 个

      第： 639 个文件
       第 639 个文件共有停留点： 0 个

      第： 640 个文件
       第 640 个文件共有停留点： 0 个

      第： 641 个文件
       第 641 个文件共有停留点： 1 个

      第： 642 个文件
       第 642 个文件共有停留点： 0 个

      第： 643 个文件
       第 643 个文件共有停留点： 0 个

      第： 644 个文件
       第 644 个文件共有停留点： 0 个

      第： 645 个文件
       第 645 个文件共有停留点： 0 个

      第： 646 个文件
       第 646 个文件共有停留点： 0 个

      第： 647 个文件
       第 647 个文件共有停留点： 0 个

      第： 648 个文件
       第 648 个文件共有停留点： 0 个

      第： 649 个文件
       第 649 个文件共有停留点： 0 个

      第： 650 个文件
       第 650 个文件共有停留点： 0 个

      第： 651 个文件
       第 651 个文件共有停留点： 0 个

      第： 652 个文件
       第 652 个文件共有停留点： 0 个

      第： 653 个文件
       第 653 个文件共有停留点： 0 个

      第： 654 个文件
       第 654 个文件共有停留点： 0 个

      第： 6

       第 14 个文件共有停留点： 0 个

      第： 15 个文件
       第 15 个文件共有停留点： 1 个

      第： 16 个文件
       第 16 个文件共有停留点： 0 个

      第： 17 个文件
       第 17 个文件共有停留点： 0 个

      第： 18 个文件
       第 18 个文件共有停留点： 0 个

      第： 19 个文件
       第 19 个文件共有停留点： 0 个

      第： 20 个文件
       第 20 个文件共有停留点： 0 个

      第： 21 个文件
       第 21 个文件共有停留点： 0 个

      第： 22 个文件
       第 22 个文件共有停留点： 0 个

      第： 23 个文件
       第 23 个文件共有停留点： 0 个

      第： 24 个文件
       第 24 个文件共有停留点： 1 个

      第： 25 个文件
       第 25 个文件共有停留点： 0 个

      第： 26 个文件
       第 26 个文件共有停留点： 0 个

      第： 27 个文件
       第 27 个文件共有停留点： 3 个

      第： 28 个文件
       第 28 个文件共有停留点： 0 个

      第： 29 个文件
       第 29 个文件共有停留点： 0 个

      第： 30 个文件
       第 30 个文件共有停留点： 0 个

      第： 31 个文件
       第 31 个文件共有停留点： 0 个

      第： 32 个文件
       第 32 个文件共有停留点： 0 个

      第： 33 个文件
       第 33 个文件共有停留点： 0 个

      第： 34 个文件
       第 34 个文件共有停留点： 2 个

      第： 35 个文件
       第 35 个文件共有停留点： 0 个

      第： 36 个文件
       第 36 个文件共有停留点： 0 个

      第： 37 个文件
       第 37

       第 249 个文件共有停留点： 1 个

      第： 250 个文件
       第 250 个文件共有停留点： 1 个

      第： 251 个文件
       第 251 个文件共有停留点： 0 个

      第： 252 个文件
       第 252 个文件共有停留点： 0 个

      第： 253 个文件
       第 253 个文件共有停留点： 3 个

      第： 254 个文件
       第 254 个文件共有停留点： 4 个

      第： 255 个文件
       第 255 个文件共有停留点： 5 个

      第： 256 个文件
       第 256 个文件共有停留点： 4 个

      第： 257 个文件
       第 257 个文件共有停留点： 0 个

      第： 258 个文件
       第 258 个文件共有停留点： 0 个

      第： 259 个文件
       第 259 个文件共有停留点： 1 个

      第： 260 个文件
       第 260 个文件共有停留点： 0 个

      第： 261 个文件
       第 261 个文件共有停留点： 0 个

      第： 262 个文件
       第 262 个文件共有停留点： 3 个

      第： 263 个文件
       第 263 个文件共有停留点： 8 个

      第： 264 个文件
       第 264 个文件共有停留点： 0 个

      第： 265 个文件
       第 265 个文件共有停留点： 2 个

      第： 266 个文件
       第 266 个文件共有停留点： 1 个

      第： 267 个文件
       第 267 个文件共有停留点： 0 个

      第： 268 个文件
       第 268 个文件共有停留点： 0 个

      第： 269 个文件
       第 269 个文件共有停留点： 0 个

      第： 270 个文件
       第 270 个文件共有停留点： 2 个

      第： 271 个文件
       第 2

      第： 1 个文件
       第 1 个文件共有停留点： 0 个

      第： 2 个文件
       第 2 个文件共有停留点： 0 个

      第： 3 个文件
       第 3 个文件共有停留点： 0 个

      第： 4 个文件
       第 4 个文件共有停留点： 0 个

      第： 5 个文件
       第 5 个文件共有停留点： 0 个

      第： 6 个文件
       第 6 个文件共有停留点： 0 个

      第： 7 个文件
       第 7 个文件共有停留点： 0 个

      第： 8 个文件
       第 8 个文件共有停留点： 0 个

      第： 9 个文件
       第 9 个文件共有停留点： 0 个

      第： 10 个文件
       第 10 个文件共有停留点： 0 个

      第： 11 个文件
       第 11 个文件共有停留点： 0 个

      第： 12 个文件
       第 12 个文件共有停留点： 0 个

      第： 13 个文件
       第 13 个文件共有停留点： 0 个

      第： 14 个文件
       第 14 个文件共有停留点： 0 个

      第： 15 个文件
       第 15 个文件共有停留点： 0 个

      第： 16 个文件
       第 16 个文件共有停留点： 0 个

      第： 17 个文件
       第 17 个文件共有停留点： 0 个

      第： 18 个文件
       第 18 个文件共有停留点： 0 个

      第： 19 个文件
       第 19 个文件共有停留点： 0 个

      第： 20 个文件
       第 20 个文件共有停留点： 1 个

      第： 21 个文件
       第 21 个文件共有停留点： 0 个

      第： 22 个文件
       第 22 个文件共有停留点： 0 个

      第： 23 个文件
       第 23 个文件共有停留点： 0 个

      第： 24 个文件
       第 24 个


      第： 70 个文件
       第 70 个文件共有停留点： 3 个
第: 179 用户共有停留点： 83 个
第： 180 个用户

      第： 0 个文件
       第 0 个文件共有停留点： 0 个

      第： 1 个文件
       第 1 个文件共有停留点： 0 个

      第： 2 个文件
       第 2 个文件共有停留点： 0 个

      第： 3 个文件
       第 3 个文件共有停留点： 0 个

      第： 4 个文件
       第 4 个文件共有停留点： 0 个
第: 180 用户共有停留点： 0 个
第： 181 个用户

      第： 0 个文件
       第 0 个文件共有停留点： 0 个

      第： 1 个文件
       第 1 个文件共有停留点： 2 个

      第： 2 个文件
       第 2 个文件共有停留点： 2 个

      第： 3 个文件
       第 3 个文件共有停留点： 2 个

      第： 4 个文件
       第 4 个文件共有停留点： 0 个

      第： 5 个文件
       第 5 个文件共有停留点： 0 个

      第： 6 个文件
       第 6 个文件共有停留点： 0 个

      第： 7 个文件
       第 7 个文件共有停留点： 1 个

      第： 8 个文件
       第 8 个文件共有停留点： 2 个

      第： 9 个文件
       第 9 个文件共有停留点： 0 个

      第： 10 个文件
       第 10 个文件共有停留点： 0 个

      第： 11 个文件
       第 11 个文件共有停留点： 0 个

      第： 12 个文件
       第 12 个文件共有停留点： 0 个

      第： 13 个文件
       第 13 个文件共有停留点： 0 个

      第： 14 个文件
       第 14 个文件共有停留点： 0 个
第: 181 用户共有停留点： 9 个
共有停留点：  18341 个


In [8]:
filename = ".\\os_file\\st_pa.pkl"
st_pa = read_q(filename)
len(st_pa)

182

In [26]:
###########################  个人停留点 list  st_pa_p list  三元组list #################################################
# 输入 st_pa   三层list    st_pa[user][file][st_p]
# st_pa_p       停留点list      三元组list       [lat, lng, time]    

i= 0
filename = ".\\os_file\\st_pa_p"+str(i)+".pkl"
st_pa_p = []
#st_p = []    #  停留点的list 

m = st_pa

if len(m[i]) == 0:            #  如果  user i 没有文件 
    sys.exit()
else:       
    for j in range(0, len(m[i])):      
                    # 对用户i的文件j, st_p[i][j],     文件
        if len(m[i][j]) == 0:       #  文件内没有stay point，
            continue
        else:      #  文件内有 stay point               #  user_file_st-p
                
            for k in range(0, len(m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                a = []
                a = m[i][j][k]
                        #print(a)
                st_pa_p.append(a)
                            
b = len(st_pa_p)
print("用户",i,"共有停留点有： ", b,"  个")

write_q(st_pa_p, filename)


用户 0 共有停留点有：  359   个


In [41]:
i= 0
filename = ".\\os_file\\st_pa_p"+str(i)+".pkl"
write_q(st_pa_p, filename)

In [29]:
filename = ".\\os_file\\st_poi_dic.pkl"
st_poi_dic = read_q(filename)
st_poi_dic

[{'id': '圆明园-心境轩',
  'location': {'lat': 40.006578000000005, 'lng': 116.320843},
  'staying': [23.749999915016815,
   23.749999915016815,
   21.71666664071381,
   19.283333325292915,
   14.750000063795596,
   17.08333329297602,
   14.750000063795596,
   11.666666585952044,
   21.833333271788433,
   17.333333424758166,
   24.250000031897798,
   22.416666615754366,
   15.583333256654441,
   41.19999998132698,
   11.666666585952044,
   21.833333271788433,
   17.333333424758166,
   13.083333258982748,
   13.450000032316893,
   30.549999880604446,
   18.499999966006726,
   10.583333282265812,
   22.13333337334916,
   20.799999927403405,
   12.016666657291353,
   10.033333342289552,
   16.433333423919976,
   34.383333313744515,
   10.283333327388391,
   15.283333291299641,
   12.966666764114052,
   18.983333370415494,
   16.383333307458088,
   13.40000006253831,
   14.933333377121016,
   11.949999975040555,
   12.166666702833027,
   20.749999968102202,
   12.050000061281025,
   16.6499999945

In [30]:
len(st_poi_dic)

3634

In [45]:
################################   st_poi   停留点 与 POI 匹配  #####################################
#  停留点   st_pa_p        三元组  [lat,lng, time]
#  POI      st_poi_dic     字典列表  [{},{},{}]
#  输出  st_poi_p      停留点对应的POI，及停留时间  [id, lat, lng, time]  
i = 0
filename = ".\\os_file\\st_poi_p"+str(i)+".pkl"   
st_poi_p = []
for i in range(0, len(st_pa_p)):
#for i in range(0, 20):
    
    st_p_m = match(st_pa_p[i], st_poi_dic)
    st_poi_p.append(st_p_m)
    
write_q(st_poi_p, filename)

In [40]:
i = 0
filename = ".\\os_file\\st_poi_p"+str(i)+".pkl"  
write_q(st_poi_p, filename)

In [46]:
st_poi_p

[['哈哈镜超市(中关村店)',
  40.009215999999995,
  116.32202099999999,
  10.766666595591232,
  'smarket'],
 ['衢州三头一掌', 40.008834, 116.322123, 25.850000018253922, 'chfood'],
 ['哈哈镜超市(中关村店)',
  40.009215999999995,
  116.32202099999999,
  10.500000055180863,
  'smarket'],
 ['风雨长廊', 40.010983, 116.298449, 24.333333405666053, 'scenic'],
 ['畅和堂', 40.005452000000005, 116.31348700000001, 12.416666687931865, 'scenic'],
 ['风雨长廊', 40.010983, 116.298449, 59.33333333116025, 'scenic'],
 ['藻园', 40.008621000000005, 116.29623899999999, 13.083333258982748, 'scenic'],
 ['风雨长廊', 40.010983, 116.298449, 10.16666668583639, 'scenic'],
 ['风雨长廊', 40.010983, 116.298449, 19.333333305548877, 'scenic'],
 ['北京大学动力中心', 39.999578, 116.324633, 15.033333452884108, 'train'],
 ['南朝书画室', 40.009499, 116.321381, 33.14999994356185, 'train'],
 ['衢州三头一掌', 40.008834, 116.322123, 11.499999985098839, 'chfood'],
 ['北京大学动力中心', 39.999578, 116.324633, 111.04999992297962, 'train'],
 ['津来居宾馆', 40.009869, 116.32210400000001, 42.80000011436641, 'eh

In [50]:
#############################  st_poi_dic_p   把 st_poi_p 四元组list 转换成 字典列表  ###################### 
#  输入  st_poi_p  根据POI 【id, lat, lng】 对停留时间进行聚集。  
#  输出 st_poi_dic_p  是 字典list        st_poi_dic_p[i][][]    第i个POI，[][]字典的操作
#    POI  {'id': '无尽',
#          'location': {'lat': 40.005973, 'lng': 116.32533400000001},
#          'staying': [68.36666659684852, 113.54999989969656]}
i = 0
filename = ".\\os_file\\st_poi_dic_p"+str(i)+".pkl"
st_poi_dic_p = []
st_poi = st_poi_p

st_poi_dic = []
for i in range(0,len(st_poi)):    #  待匹配的 st_poi list
    num = 0
    for j in range(0, len(st_poi_dic) ):        # 遍历已有列表
        
        if  st_poi[i][0] == st_poi_dic[j]["id"] and st_poi[i][1] == st_poi_dic[j]["location"]["lat"]  and  \
        st_poi[i][2] == st_poi_dic[j]["location"]["lng"]:       # 如果列表中已有该字典条目
            
            st_poi_dic[j]["staying"].append( st_poi[i][3])
            break
        else:
            num += 1
            #continue
    if num == len(st_poi_dic):        # 如果 现有st_poi_dic  list  中没有，则为该POI 添加 一个新的字典条目
        a = {}
        a["id"] = st_poi[i][0]
        a["location"] = {"lat": st_poi[i][1], "lng": st_poi[i][2]}
        a["staying"] = [st_poi[i][3]]
        a["category"] = st_poi[i][4]
        st_poi_dic.append(a)
    #else:
        #continue
st_poi_dic_p = st_poi_dic
write_q(st_poi_dic_p, filename)

In [52]:
st_poi_dic_p

[{'id': '哈哈镜超市(中关村店)',
  'location': {'lat': 40.009215999999995, 'lng': 116.32202099999999},
  'staying': [10.766666595591232, 10.500000055180863],
  'category': 'smarket'},
 {'id': '衢州三头一掌',
  'location': {'lat': 40.008834, 'lng': 116.322123},
  'staying': [25.850000018253922, 11.499999985098839],
  'category': 'chfood'},
 {'id': '风雨长廊',
  'location': {'lat': 40.010983, 'lng': 116.298449},
  'staying': [24.333333405666053,
   59.33333333116025,
   10.16666668583639,
   19.333333305548877],
  'category': 'scenic'},
 {'id': '畅和堂',
  'location': {'lat': 40.005452000000005, 'lng': 116.31348700000001},
  'staying': [12.416666687931865],
  'category': 'scenic'},
 {'id': '藻园',
  'location': {'lat': 40.008621000000005, 'lng': 116.29623899999999},
  'staying': [13.083333258982748],
  'category': 'scenic'},
 {'id': '北京大学动力中心',
  'location': {'lat': 39.999578, 'lng': 116.324633},
  'staying': [15.033333452884108, 111.04999992297962],
  'category': 'train'},
 {'id': '南朝书画室',
  'location': {'lat':

In [54]:
#################################   基于字典 建立 st_poi_tree 分类树   ##########################
"""#  大类 是 字典，  enter["scenic"]； 
#  小类 是 列表，  scenic[{},{},{}];   scenic[2]["staying"]
#  st_poi_dic_p  是  字典list， st_poi = {id: , location: , staying: , category: }"""
i = 0
filename = ".\\os_file\\st_poi_tree_p"+str(i)+".pkl"
st_poi_tree_p = {}
st_poi_dic = st_poi_dic_p

path = ".\\POI_base\\POI\\"                               # 根目录
class1_list = os.listdir(path)                            # 大类列表
createVar = locals()    # 动态生成变量

st_poi_tree = {}

#print("所有大类： ", class1_list)  
for i in range(0, len(class1_list)):
    #print("当前大类： ", class1_list[i])        
    class2_path = path + class1_list[i]                   # 大类路径 ， 每个大类
    class2_list = os.listdir(class2_path)                 # 小类列表
    #print("小类列表： ", class2_list) 
    
    name1 = os.path.splitext(class1_list[i])[0]      # 取出大类名    去掉后缀名
    createVar[name1] = {}   # 创建大类字典
    
    for j in range(0, len(class2_list)):
        class2_file = class2_path +"\\"+ class2_list[j]     # 小类路径 ， 每个小类
        #print("大类： ",class1_list[i],"下的当前小类： ",  class2_list[j])
        #print("小类文件目录： ", class2_file)
        name2 = os.path.splitext(class2_list[j])[0]      # 取出小类名
        createVar[name2] = []      #创建小类  列表
        
        
        for k in range(0, len(st_poi_dic)):         # 对st_poi_dic 进行分类
            
            if st_poi_dic[k]["category"] == name2:
                createVar[name2].append(st_poi_dic[k])
        
        
        
        createVar[name1][name2] = createVar[name2]    # 把小类字典 嵌入 大类字典
        
    st_poi_tree[name1] = createVar[name1]       # 小类遍历完后，把大类字典，嵌入树
        
st_poi_tree_p = st_poi_tree       
write_q(st_poi_tree_p, filename)


In [55]:
st_poi_tree_p

{'enter': {'bar': [],
  'bath': [{'id': '聚美足道',
    'location': {'lat': 40.010055, 'lng': 116.32128600000001},
    'staying': [13.983333270298317],
    'category': 'bath'},
   {'id': '北京锡华商务酒店-康体中心',
    'location': {'lat': 39.995405, 'lng': 116.30925500000001},
    'staying': [17.416666651843116],
    'category': 'bath'}],
  'bbulid': [{'id': '北京大学附属小学-体育馆',
    'location': {'lat': 40.000085, 'lng': 116.32512},
    'staying': [139.11666667554528,
     13.41666662832722,
     336.84999998426065,
     124.30000007618219,
     215.64999994123355,
     212.31666662497446,
     105.74999999371357,
     308.3833333477378,
     181.0833333141636,
     180.0,
     55.23333335760981,
     182.9333334218245,
     185.23333324701525,
     98.53333328501321,
     228.51666661910713,
     30.6999999366235,
     18.04999996558763,
     49.733333266340196,
     229.2333333066199,
     252.41666672634892,
     133.43333328142762,
     303.9500000048429,
     305.19999993033707,
     21.84999998426065

In [75]:
############################  统计语义类型占比  __个人  #################
#  输入 st_poi_tree_p
#  输出 st_poi_pro_p
i = 0
filename = ".\\os_file\\st_poi_pro_p"+str(i)+".pkl"
tree = st_poi_tree_p
pro = copy.deepcopy(tree)

a = 0
for item1 in tree:
    
    for item2 in tree[item1]:
        
        st = tree[item1][item2]
       
        for i in range(len(st)):    # 小类下的每一个 POI，
            
            stay = st[i]["staying"]
            a += len(stay)
            #for j in range(len(stay)):
print("该用户共有：",a,"个访问点")             
        

for item1 in tree:
    
    for item2 in tree[item1]:
        
        st = tree[item1][item2]
        b = 0       
        for i in range(len(st)):    # 小类下的每一个 POI，
            
            stay = st[i]["staying"]
            b += len(stay)
            #for j in range(len(stay)):
        pro[item1][item2] = round(b/a, 5)
st_poi_pro_p = pro

write_q(st_poi_pro_p, filename)


该用户共有： 359 个访问点


In [11]:
filename = ".\\os_file\\st_poi_dic.pkl"
st_poi_dic = read_q(filename)
len(st_poi_dic)

3634

In [12]:
##############################   单文件的停留点 单轨迹  ####################
## 输入 st_pa   所有停留点  三层list  [user][file][st_p]
## 输出 st_poi_f  五元组list   [id, lat, lng,time, category]


i = 0
j = 47    # 第48个文件
filename = ".\\os_file\\st_poi_f_"+str(i)+"_"+str(j)+".pkl"

st_pa_f = []
#st_p = []    #  停留点的list 

m = st_pa

if len(m[i]) == 0:            #  如果  user i 没有文件 
    print("该用户没有轨迹文件")
    sys.exit()
else:       
    if len(m[i][j]) == 0:       #  文件内没有stay point，
        print("该文件内没有停留点")
        sys.exit()
    else:      #  文件内有 stay point               #  user_file_st-p
        for k in range(0, len(m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
            a = []
            a = m[i][j][k]
            st_pa_f.append(a)
            
b = 0                            
b = len(st_pa_f)
print("用户： ",i," 在文件： ",j," 中共有停留点有： ", b,"  个")

#################  匹配 st_pa_f ----> st_poi_f ############### 
st_poi_f = []
for i in range(0, len(st_pa_f)):
#for i in range(0, 20):
    
    st_p_m = match(st_pa_f[i], st_poi_dic)
    st_poi_f.append(st_p_m)

write_q(st_poi_f, filename)


用户：  0  在文件：  47  中共有停留点有：  14   个


In [13]:
st_poi_f

[['无尽', 40.005973, 116.32533400000001, 65.81666663987562, 'scenic'],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 19.833333265269175, 'scenic'],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 105.74999999371357, 'bbulid'],
 ['格林豪泰酒店(首都机场航站楼店)', 40.082075, 116.59334, 10.333333286689594, 'ehotel'],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 16.333333337679505, 'scenic'],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 22.5, 'scenic'],
 ['万春园丙区', 40.007911, 116.31916399999999, 10.3333334333729, 'scenic'],
 ['生冬室', 40.009062, 116.315025, 342.0833333779592, 'scenic'],
 ['生冬室', 40.009062, 116.315025, 136.0833333141636, 'scenic'],
 ['生冬室', 40.009062, 116.315025, 407.08333325455897, 'scenic'],
 ['颐和园-治镜阁遗址', 39.997733000000004, 116.270547, 12.166666692355648, 'scenic'],
 ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm'],
 ['宏都商场', 39.908799, 116.40641000000001, 38.08333324617706, 'smarket'],
 ['天安门广场', 39.909652, 116.404177, 11.666666596429422, 'scenic']]

In [15]:
filename = ".\\os_file\\st_poi_pro_p0.pkl"
st_poi_pro_p = read_q(filename)
filename = ".\\os_file\\st_poi_pro.pkl"
st_poi_pro = read_q(filename)
print(st_poi_pro_p["enter"]["bath"], st_poi_pro["enter"]["bath"])

0.00557 0.03082


In [20]:
st_poi_pro

{'enter': {'bar': 0.00411,
  'bath': 0.03082,
  'bbulid': 0.13862,
  'cinema': 0.00016,
  'farm': 0.00302,
  'ktv': 0.00439,
  'scenic': 0.1617,
  'tea': 0.01481},
 'food': {'buffet': 0.0182,
  'chfood': 0.01716,
  'ffood': 0.08878,
  'hotpot': 0.02336,
  'jkcui': 0.00724,
  'sweet': 0.01069,
  'wfood': 0.00642},
 'hotel': {'ehotel': 0.12519,
  'fivestar': 0.0,
  'fourstar': 0.0,
  'raccom': 0.0,
  'threestar': 0.0},
 'school': {'kgartern': 0.0,
  'mschool': 0.0108,
  'pschool': 0.02506,
  'train': 0.13522,
  'univer': 0.03361},
 'shop': {'bookstore': 0.00554,
  'fmarket': 0.01623,
  'mall': 0.00132,
  'smarket': 0.11756}}

In [1]:
##############################  semantic importance  ##################
## 输入 st_poi_f  五元组-两维list   [id, lat, lng,time, category] ;    st_poi_pro_p, st_poi_pro
## 输出 si_l_f    六元组-两维list   [id, lat, lng,time, category,level]
                  # 输出文件 si_l_f_0_47.pkl   


i = 0
j = 47
filename = ".\\os_file\\si_l_f_"+str(i)+"_"+str(j)+".pkl"

f = st_poi_f
pro_p = st_poi_pro_p
pro = st_poi_pro

si_l_f = []
for i in range(len(f)):
    
    ca = f[i][4]
    
    flag = 0
    
    for item1 in pro:
        
        if flag == 1:
            break
        else:
            
            for item2 in pro[item1]:
                if flag == 1:
                    break
                else:
                    if ca == item2:         # 判断语义类型匹配
                        flag = 1
                        a = 0
                        a = pro[item1][item2]
                        b = 0
                        b = pro_p[item1][item2]
                        
                        if a == 0:
                            print("st_poi_pro 没有该类型的停留点")
                            
                        c=0
                        c = b * math.log(1/a)
                        
                        d = [f[i][0],f[i][1],f[i][2],f[i][3],f[i][4],c]
                        si_l_f.append(d)
                    
write_q(si_l_f, filename)


NameError: name 'st_poi_f' is not defined

In [35]:
############################  查看 semantic importance 的值   #############
#  si_l_f   [id, lat, lng,time, category,level]

al = []

for i in range(len(si_l_f)):
    al.append(si_l_f[i][5])
    
len(al)

14

In [42]:
###########################  si_h_f 语义树高度   #######################
## 输入 si_l_f， o_l, o_r   
## 输出 si_h_f     六元组-两维list   [id, lat, lng,time, category,height]
                  # 输出文件 si_h_f_0_47.pkl 
i = 0
j = 47
filename = ".\\os_file\\si_h_f_"+str(i)+"_"+str(j)+".pkl"

o_l = 1
o_r = 3
    
al = []
for j in range(len(si_l_f)):
    al.append(si_l_f[j][5])
w = max(al)

si_h_f = []
for k in range(len(si_l_f)):
    a = si_l_f[k][5]
    b = a/w
    c = 1-b
    
    l = c * o_l + b *o_r
    h = round(l)
    
    d = []
    d = [si_l_f[k][0],si_l_f[k][1],si_l_f[k][2],si_l_f[k][3],si_l_f[k][4],h]
    si_h_f.append(d)

write_q(si_h_f, filename)

si_h_f

[['无尽', 40.005973, 116.32533400000001, 65.81666663987562, 'scenic', 2],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 19.833333265269175, 'scenic', 2],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 105.74999999371357, 'bbulid', 3],
 ['格林豪泰酒店(首都机场航站楼店)', 40.082075, 116.59334, 10.333333286689594, 'ehotel', 2],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 16.333333337679505, 'scenic', 2],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 22.5, 'scenic', 2],
 ['万春园丙区', 40.007911, 116.31916399999999, 10.3333334333729, 'scenic', 2],
 ['生冬室', 40.009062, 116.315025, 342.0833333779592, 'scenic', 2],
 ['生冬室', 40.009062, 116.315025, 136.0833333141636, 'scenic', 2],
 ['生冬室', 40.009062, 116.315025, 407.08333325455897, 'scenic', 2],
 ['颐和园-治镜阁遗址',
  39.997733000000004,
  116.270547,
  12.166666692355648,
  'scenic',
  2],
 ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm', 1],
 ['宏都商场', 39.908799, 116.40641000000001, 38.08333324617706, 'smarket', 1],
 ['天安门广场', 39.909652, 116.404177, 11.66666659

In [40]:
al = []
for j in range(len(si_h_f)):
    al.append(si_h_f[j][5])
al

[2.4752691069765325,
 2.4752691069765325,
 3.0,
 1.8089180643487326,
 2.4752691069765325,
 2.4752691069765325,
 2.4752691069765325,
 2.4752691069765325,
 2.4752691069765325,
 2.4752691069765325,
 2.4752691069765325,
 1.0452486498926978,
 1.3333473060607341,
 2.4752691069765325]

In [77]:
filename = ".\\os_file\\st_poi_tree.pkl"
st_poi_tree = read_q(filename)

In [78]:
############################  统计语义类型占比  _ 总体  #################
#  输入 st_poi_tree
#  输出 st_poi_pro
i = 0
filename = ".\\os_file\\st_poi_pro.pkl"
tree = st_poi_tree
pro = copy.deepcopy(tree)

a = 0
for item1 in tree:
    
    for item2 in tree[item1]:
        
        st = tree[item1][item2]
       
        for i in range(len(st)):    # 小类下的每一个 POI，
            
            stay = st[i]["staying"]
            a += len(stay)
            #for j in range(len(stay)):
print("所有用户共有：",a,"个访问点")             
        

for item1 in tree:
    
    for item2 in tree[item1]:
        
        st = tree[item1][item2]
        b = 0       
        for i in range(len(st)):    # 小类下的每一个 POI，
            
            stay = st[i]["staying"]
            b += len(stay)
            #for j in range(len(stay)):
        pro[item1][item2] = round(b/a, 5)
        
st_poi_pro = pro

write_q(st_poi_pro, filename)


所有用户共有： 18237 个访问点


In [79]:
filename = ".\\os_file\\st_poi_pro.pkl"
st_po = read_q(filename)
st_po

{'enter': {'bar': 0.00411,
  'bath': 0.03082,
  'bbulid': 0.13862,
  'cinema': 0.00016,
  'farm': 0.00302,
  'ktv': 0.00439,
  'scenic': 0.1617,
  'tea': 0.01481},
 'food': {'buffet': 0.0182,
  'chfood': 0.01716,
  'ffood': 0.08878,
  'hotpot': 0.02336,
  'jkcui': 0.00724,
  'sweet': 0.01069,
  'wfood': 0.00642},
 'hotel': {'ehotel': 0.12519,
  'fivestar': 0.0,
  'fourstar': 0.0,
  'raccom': 0.0,
  'threestar': 0.0},
 'school': {'kgartern': 0.0,
  'mschool': 0.0108,
  'pschool': 0.02506,
  'train': 0.13522,
  'univer': 0.03361},
 'shop': {'bookstore': 0.00554,
  'fmarket': 0.01623,
  'mall': 0.00132,
  'smarket': 0.11756}}